In [4]:
from tqdm import tqdm
from path import Path
import json
import time
from pprint import pprint

import numpy as np
import pandas as pd
import re


basedir=r'/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/'
os.chdir(basedir)

In [8]:
from path import Path
from collections import OrderedDict

# import makehuman-cmd

https://bitbucket.org/duststorm01/makehuman-commandline

In [2]:
# from .mh_helpers import clean, short_hash, clean_modifier

mhpath = Path(os.path.abspath("../vendor/makehuman-commandline/makehuman"))

#===============================================================================
# Import Makehuman resources, needs to be with makehuman dir as current dir
#===============================================================================

appcwd = os.path.abspath(os.curdir)
sys.path.append(mhpath)
sys.path.append(appcwd)
sys.path.append('.')

def getHuman():
    """Load a human model with modifiers."""
    with mhpath:
        # maxFaces *uint* Number of faces per vertex (pole), None for default (min 4)
        human = Human(files3d.loadMesh(
            getpath.getSysDataPath("3dobjs/base.obj"),
            maxFaces=5))
        # load modifiers onto human
        humanmodifier.mods_loaded = False
        modifiers = humanmodifier.loadModifiers(
            getpath.getSysDataPath('modifiers/modeling_modifiers.json'), human)
        return human

with mhpath:
    import makehuman
    oldpath = os.sys.path
    makehuman.set_sys_path()
    # make makehuman paths absolute by going through newest paths and making abs
    for i in range(len(os.sys.path)):
        p = os.sys.path[i]
        if p[0:2] == './':
            os.sys.path[i] = os.path.join(
                os.path.abspath('.'), p.replace('./', ''))
        else:
            break

    makehuman.init_logging()
    #import image_pil as image_lib
    #
    import proxy as mhproxy
    import humanargparser
    import targets as mhtargets
    from human import Human
    import files3d
    import getpath
    import humanmodifier
    from core import G
    import headless
    import autoskinblender
    import export
    
    # Init console app
    with mhpath:
        G.app = headless.ConsoleApp()
    G.app.selectedHuman = human = getHuman()
    headless.OBJExporter = None
    headless.MHXExporter = None
    headless.MhxConfig = None
    humanargparser.mods_loaded = False

def assignModifierValues(human, valuesDict):
    _tmp = human.symmetryModeEnabled
    human.symmetryModeEnabled = False
    for mName, val in valuesDict.items():
        try:
            human.getModifier(mName).setValue(val)
        except:
            pass
    human.updateMacroModifiers()
    human.applyAllTargets()
    human.symmetryModeEnabled = _tmp
    return human

Initialized logging
Loading binary mesh data/3dobjs/base.npz.
Loading material from file data/skins/default.mhmat
Loading binary mesh data/3dobjs/base.npz.
Loading material from file data/skins/default.mhmat
Loading modifiers from data/modifiers/modeling_modifiers.json
Attempting to load targets from NPZ file.
1258 targets loaded from NPZ file succesfully.
Loaded 249 modifiers from file data/modifiers/modeling_modifiers.json
Loaded 198 modifier descriptions from file data/modifiers/modeling_modifiers_desc.json


In [9]:
# import proxies
def _listDataFiles(foldername,
                   extensions,
                   onlySysData=False,
                   recursive=True):
    with mhpath:  # sadly makehuman seems hardcoded
        if onlySysData:
            paths = [getpath.getSysDataPath(foldername)]
        else:
            paths = [getpath.getDataPath(foldername),
                     getpath.getSysDataPath(foldername)]
    return list(getpath.search(paths, extensions, recursive))

def clean(s):
    """Remove invalid characters."""
    s = re.sub('[^0-9a-zA-Z_]', '_', s)
    return s

with mhpath:
    mhproxy.ProxyTypes
    proxies = OrderedDict()
    for proxyType in mhproxy.ProxyTypes:
        files = list(_listDataFiles(proxyType.lower(),
                                         ['.proxy', '.mhclo']))
        for f in files:
            if proxyType not in proxies.keys():
                proxies[proxyType] = OrderedDict()
            filesname = clean(os.path.splitext(os.path.basename(f))[0])
            proxies[proxyType][filesname] = f

In [10]:
# old dupicate imports
import wrap_mh
import proxy
from human import Human
import getpath
from pprint import pprint
import files3d
import humanargparser
import humanmodifier

mhpath=wrap_mh.mhpath


# add mh lic info
from makehuman import LicenseInfo
mh_licence=LicenseInfo()

In [11]:
# silence mh
import logging
logging.getLogger().setLevel(logging.CRITICAL)

# Make random outputs for a VAE

TODO
- tidy
- I'm saving -morphTargets, so flip it around next time I generate data


## Export all modifiers as obj's


In [12]:
# choose proxymesh
proxyname= None #'female1605'
print 'Proxies to choose'
pprint(proxies['Proxymeshes'].keys())
export_helpers=False

if proxyname:
    proxyfile=proxies['Proxymeshes'][proxyname]
else:
    proxyfile=None
timestr = time.strftime("%Y%m%d-%H%M%S")
runname=timestr+'_'+str(proxyname)
print 'runname',runname

basehuman=getHuman()
with mhpath:
    if proxyname:
        pxy = proxy.loadTextProxy(basehuman, proxyfile, type='Proxymesh')
    else:
        pxy = None
if proxyname:
    proxyMetadata={
        'name': pxy.name,
        'description': pxy.description,
        'file': os.path.basename(pxy.file),
        'license': pxy.license.asDict(),
        'vertexBoneWeights_file': pxy.vertexBoneWeights_file,
        'version': pxy.version,
        'obj_file': os.path.basename(pxy.obj_file),
        'basemesh': pxy.basemesh,
    }
else:
    proxyMetadata = {}

Proxies to choose
[u'female1605',
 u'female_generic',
 u'female_muscle_13442',
 u'male1591',
 u'male_generic',
 u'male_muscle_13290',
 u'proxy10k',
 u'human',
 u'proxy5k',
 u'proxy741',
 u'proxymesh_test']
runname 20161117-193527_None


In [13]:
# choose rig
rigfile='data/rigs/default_no_toes.mhskel'
# some poses involve balled hands etc, so I choose this one with ~130 bones as opposed to the 30

# list
with mhpath:
    paths = [getpath.getSysDataPath('rigs')]
    files=getpath.search(paths, ['.mhskel'], False)
    print "Available rigs:"
    print "\n".join(['  %s' % r for r in files])


    rigfile = getpath.findFile(rigfile,
                               searchPaths = [getpath.getSysDataPath(),
                                              getpath.getSysPath()])
    print 'rig', rigfile

Available rigs:
  data/rigs/cmu_mb.mhskel
  data/rigs/default.mhskel
  data/rigs/default_no_toes.mhskel
  data/rigs/game_engine.mhskel
rig data/rigs/default_no_toes.mhskel


In [14]:
from path import Path
import arrow
import uuid

# choose a new outdir
os.chdir(basedir)
outputdir = os.path.abspath(os.path.join("output",runname+"_vae_data"))
print ('outputdir',outputdir)
try:
    os.makedirs(outputdir)
except Exception as exc:
    print('makedir',exc)
    pass

outputdir = Path(outputdir)

('outputdir', '/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/output/20161117-193527_None_vae_data')


In [15]:
# some consistent scheme for modifier values (alphebetical order) 
# and scale them to 0-1
# is_metadetail = np.array(['macrodetails' in s for s in target_dict.values()])*1
from collections import OrderedDict
human = getHuman()
target_dict = OrderedDict(enumerate(sorted([m.fullName for m in human.modifiers])))
target_dict_rev = OrderedDict((v,k) for k,v in target_dict.items())

mins = np.array([i[1] for i in sorted([(m.fullName,m.getMin()) for m in human.modifiers])])
maxs = np.array([i[1] for i in sorted([(m.fullName,m.getMax()) for m in human.modifiers])])

def modifiers2params(modifierValues):
    """Shift the params from [-1,1] to [0,1]"""
    v=np.array([modifierValues[m] for m in target_dict_rev.keys()])
    return (v-mins)/(maxs-mins)


def params2modifiers(params):
    r=params*(maxs-mins)+mins
    # shuffle the order like in the random plugin for makehuman
    i=zip(target_dict_rev.keys(),r)
    np.random.shuffle(i)
    return OrderedDict(i)
    
def rand_modifier_values():
    r=np.random.random((len(human.modifiers)))
    return params2modifiers(r)

modifier_values=rand_modifier_values()
mv=np.array(modifier_values.values())
# test it started from [-1,1]
assert (mv<=0).any()
assert (mv>=-1).all()
p=modifiers2params(modifier_values)
# test it's moved to [0,1]
assert (p>=0).all()
assert (p<=1).all()
# check it's invertible
assert (modifiers2params(params2modifiers(p))==p).all(), 'should be invertible'

In [16]:
# make sure defaults are right
modifier_values = params2modifiers(np.ones_like(p)/2)
for m in human.modifiers:
    v = modifier_values[m.fullName]
    assert v<=m.getMax()
    assert v>=m.getMin()
    try:
        assert v==m.getDefaultValue(),'%s'%m
    except:
        print(m,m.getMin(),m.getDefaultValue(),m.getMax())

(EthnicModifier macrodetails/African, 0.0, 0.3333333333333333, 1.0)
(UniversalModifier head/head-invertedtriangular, 0.0, 0, 1.0)
(EthnicModifier macrodetails/Caucasian, 0.0, 0.3333333333333333, 1.0)
(UniversalModifier head/head-oval, 0.0, 0, 1.0)
(UniversalModifier head/head-diamond, 0.0, 0, 1.0)
(EthnicModifier macrodetails/Asian, 0.0, 0.3333333333333333, 1.0)
(UniversalModifier head/head-triangular, 0.0, 0, 1.0)
(UniversalModifier head/head-square, 0.0, 0, 1.0)
(UniversalModifier head/head-round, 0.0, 0, 1.0)
(UniversalModifier head/head-rectangular, 0.0, 0, 1.0)


In [17]:
# write metadata
target_dict_file = outputdir.joinpath('metadata.json')

metadata = {
    "name"      : runname,
    "nvertex"   : basehuman.mesh.coord.shape,
    "proxyMetadata": json.dumps(proxyMetadata),
    "target_dict": target_dict,
    "date": arrow.utcnow().format()
}

json.dump(metadata, open(target_dict_file,'w'))
print(target_dict_file)
metadata

# define x and y files
X_file = outputdir.joinpath('X_train.hdf5')
y_file = outputdir.joinpath('y_train.hdf5')

# write headers
import tables
with tables.open_file(X_file, 'w') as xfo:
    data = basehuman.mesh.coord
    atom = tables.Atom.from_dtype(data.dtype)
    # create an expandable array
    data_storage = xfo.create_earray(xfo.root, 'data', atom, (0,data.shape[0],data.shape[1]))

# process the results
values = dict([(m.fullName,m.getValue()) for m in basehuman.modifiers])
data = np.array([values.get(t,0) for t in target_dict.values()],dtype=np.float32)
data
with tables.open_file(y_file, 'w') as xfo:
    atom = tables.Atom.from_dtype(data.dtype)
    # create an expandable array
    data_storage = xfo.create_earray(xfo.root, 'data', atom, (0,data.shape[0]))

/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/output/20161117-193527_None_vae_data/metadata.json


In [18]:
# make base human model
with mhpath: 
    basehuman = getHuman()
    
    # rig
    if rigfile:
        humanargparser.addRig(basehuman,rigfile)

    if proxyname:
        # load proxy
        pxy = proxy.loadTextProxy(basehuman, proxyfile, type='Proxymesh')
        mesh,obj = pxy.loadMeshAndObject(basehuman)
        basehuman.setProxy(pxy)

        # _adaptProxyToHuman
        mesh = obj.getSeedMesh() #  just gets proxy mesh
        pxy.update(mesh) # mesh change cords and calc normals
        mesh.update() # calc norms, coorgs and tangents
    
    randomValues=rand_modifier_values()
    params=modifiers2params(randomValues)
    params = np.ones_like(params)/2 # grab middle values
    randomValues=params2modifiers(params)
    human.updateMacroModifiers()
    basehuman = assignModifierValues(basehuman, randomValues)
    
    mesh = basehuman.meshData
    group_mask = np.ones(len(mesh._faceGroups), dtype=bool)
    face_mask = group_mask[mesh.group]
    basehuman._staticFaceMask = face_mask
    basehuman.meshData.changeFaceMask(basehuman.staticFaceMask)
    basehuman.meshData.updateIndexBufferFaces()
    basehuman.changeVertexMask(None)

In [19]:
# make the edge cases mixed with some random ones
nb_targets=len(target_dict)
edge_params = np.ones((2*nb_targets+1,nb_targets))/2
for i in range(nb_targets):
    edge_params[i,i]=1.0
    edge_params[nb_targets+i,i]=0.0
    
# add some random ones and shuffle
edge_params = np.concatenate([edge_params,np.random.random(edge_params.shape)])
np.random.shuffle(edge_params)

for params in tqdm(edge_params, leave=True):
    # fresh args but re-use the old human to save time initialising
    with mhpath: 

        human = getHuman()

        if proxyname:
            # load proxy
            pxy = proxy.loadTextProxy(human, proxyfile, type='Proxymesh')
            mesh,obj = pxy.loadMeshAndObject(human)
            human.setProxy(pxy)

            # _adaptProxyToHuman
            mesh = obj.getSeedMesh() #  just gets proxy mesh
            pxy.update(mesh) # mesh change cords and calc normals
            mesh.update() # calc norms, coorgs and tangents

        randomValues = params2modifiers(params)
        human = assignModifierValues(human, randomValues)
        morphTarget = -(basehuman.mesh.coord-human.mesh.coord)       
        
        
        # append the results to a file as float32
        with tables.open_file(X_file, 'a') as xfo:
            xfo.root.data.append([morphTarget])
        with tables.open_file(y_file, 'a') as xfo:
            xfo.root.data.append([params])

|##########| 998/998 100% [elapsed: 1:01:53 left: 00:00,  0.27 iters/sec]

In [31]:
rv=np.array([m[1] for m in sorted([(m.fullName,m.getValue()) for m in human.modifiers])])
rv2=np.array([m[1] for m in sorted(randomValues.items())])

In [42]:
len(human.targetsDetailStack)

322

In [21]:
# now make random targets and save to file
for t in tqdm(range(16000), leave=True):
    # fresh args but re-use the old human to save time initialising
    with mhpath: 

        human = getHuman()

        if proxyname:
            # load proxy
            pxy = proxy.loadTextProxy(human, proxyfile, type='Proxymesh')
            mesh,obj = pxy.loadMeshAndObject(human)
            human.setProxy(pxy)

            # _adaptProxyToHuman
            mesh = obj.getSeedMesh() #  just gets proxy mesh
            pxy.update(mesh) # mesh change cords and calc normals
            mesh.update() # calc norms, coorgs and tangents

        # set random targets
        randomValues=rand_modifier_values()
        params=modifiers2params(randomValues)
        human = assignModifierValues(human, randomValues)
        
        # process the results
#         values = dict([(m.fullName,m.getValue()) for m in human.modifiers])
#         params = np.array([values.get(t,0) for t in target_dict.values()],dtype=np.float32)
        # normalise the params? shift to 0-1 except for metadata ones which already are
#         params = normalise_params(params)
        morphTarget = -(basehuman.mesh.coord-human.mesh.coord)       
        
        
        # append the results to a file as float32
        with tables.open_file(X_file, 'a') as xfo:
            xfo.root.data.append([morphTarget])
        with tables.open_file(y_file, 'a') as xfo:
            xfo.root.data.append([params])

|##--------| 4276/16000  26% [elapsed: 5:33:36 left: 15:14:42,  0.21 iters/sec]

KeyboardInterrupt: 

# export skeleton

In [39]:
# export SKELETON
skeleton=basehuman.getSkeleton()


import json
from collections import OrderedDict
import os

import transformations as tm

## add to metadata
jsondata = OrderedDict()
if "metadata" not in jsondata:
    jsondata["metadata"]={}
skeletonMetadata=jsondata["metadata"]["skeleton"]={ "name": skeleton.name,
                         "version": skeleton.version,
                         "description": skeleton.description,
                         "plane_map_strategy": skeleton.plane_map_strategy,
                         "license": skeleton.license.asDict(),
                       }

lastPos=skeleton.getBones()[0].headPos
# bones
bones = []
for bone in skeleton.getBones():
    bonedef = {}
    bonedef["name"]=bone.name
    
    if bone.parent:
        bonedef["parent"] = bone.parent.index
    else:
        bonedef["parent"] = -1
    
    # distortion form res, [0,0,0,1] is null in threejs
    bonedef["rotq"] = [0, 0, 0, 1]
    
    # tail position relative to parent
    if bone.parent:
        bonedef["pos"]= (bone.tailPos-bone.parent.tailPos).tolist()
    else:
        bonedef["pos"]= bone.tailPos.tolist()
        # adjust for offset in center of geom? nope
        # bonedef["pos"][1]-=human.getBoundingBox().mean(0)[1]
    
    # scale
    bonedef["scl"]= [skeleton.scale,skeleton.scale,skeleton.scale] 

    bones.append(bonedef)
jsondata["bones"] = bones

In [40]:
world_tail_pos={}
for bone in bones:
    pos=np.array(bone["pos"])
    if bone["parent"]!=-1:
        parent = bones[bone["parent"]]
        pos+=world_tail_pos[parent['name']]
    world_tail_pos[bone['name']]=pos
df=pd.DataFrame.from_dict(world_tail_pos,orient="index")
df.columns=['x','y','z']

In [41]:
# Save weights and inds
# the data is indexes by bones but we need to reorder it by vertex, so first compile the data

# get weights for the proxied mesh
if proxyname:
    vertex_weights=basehuman.mesh.getVertexWeights(basehuman.proxy.getVertexWeights(skeleton.getVertexWeights()))
else:
    vertex_weights=basehuman.mesh.getVertexWeights(skeleton.getVertexWeights())

from collections import defaultdict
vwdata=defaultdict(dict)
for bone in skeleton.getBones():
    if bone.name in vertex_weights.data:
        for vert,w in zip(*vertex_weights.data[bone.name]):
            vwdata[vert][bone.index]=w


# now we can export them
LOST_CONNECTIONS=0

jsondata["influencesPerVertex"] = influencesPerVertex = 4 # max unless I change skinnedMesh
skinIndices=[]
skinWeights=[]
for vert in range(vertex_weights.vertexCount):
    vw=vwdata[vert].items()
    a=np.array(vw).T
    a.sort(-1) # sort by weight
    
    if a.shape[1]>influencesPerVertex:
        LOST_CONNECTIONS+=a.shape[1]-influencesPerVertex
    
    # pad and crop
    pca=np.zeros((2,influencesPerVertex), dtype=float)
    min_size=min(pca.shape[1],a.shape[1])
    pca[:2,:min_size]=a[:2,:min_size]
    
    skinIndices+=pca[0,:].tolist()
    skinWeights+=pca[1,:].tolist()
jsondata["skinWeights"] = skinWeights
jsondata["skinIndices"] = skinIndices

assert len(skinWeights)==len(skinIndices),'Should have equal number of inds and weights'
assert vertex_weights.vertexCount*influencesPerVertex==len(skinIndices),'should have 4 times as many inds as vertices'
assert max(skinIndices)<vertex_weights.vertexCount, 'vertex index should not refer to more vertices than we have'

# export base human while masking helpers

In [42]:
# combine base and morphs while giving helpers a diff index

## TODO make this iterative

# The base object has invisible groups but they are not parsed by json loader so I need to
# change it to add materials for each group, and a matrial index. This will let me hide them in three js
# at the moment they are just groups

# inputs
if proxyfile:
    # use the proxy obj as it might have groups in which we want
    infile=mhpath+'/'+proxyfile.replace('.proxy','.obj')
else:
    infile=mhpath+'/data/3dobjs/base.obj'
#     infile=baseargs["outputs"][0]
print infile
from collections import Counter
from wrap_mh.convert.convert_obj_three import (ALIGN, SHADING, BAKE_COLORS, TRUNCATE, TEMPLATE_FILE_ASCII, SCALE, 
    generate_materials_string, generate_morph_colors, extract_material_colors, generate_uv, generate_normal,
    generate_face, generate_face, generate_color_decimal, generate_vertex, generate_morph_targets, get_name,
    parse_obj)

morphfilesa=[]
morphfilesa
morphfiles=' '.join(morphfilesa)
colorfiles=''
outfile=os.path.abspath(os.path.join("output","json",runname+"_mh2obh2json.json"))

# start by parsing base mh obj
faces, vertices, uvs, normals, materials, mtllib = wrap_mh.convert.convert_obj_three.parse_obj(infile)

/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/vendor/makehuman-commandline/makehuman/data/3dobjs/base.obj


In [43]:
################################
# modify the base obj
#################################

# order unique groups by how common they are
groupCounts = Counter([f['group'] for f in faces])
groups = [g[0] for g in groupCounts.most_common()]

# set material index
for i in range(len(faces)):
    group = faces[i]["group"]
    material = groups.index(group)
    faces[i]["material"] = material

# and make those materials
materials={}
for group in groups:
    materialInd = groups.index(group)
    materials[group]=materialInd
#     {
#         "DbgColor" : 15658734,
#         "DbgIndex" : materialInd,
#         "DbgName" : group,
#         "colorDiffuse" : [0.5903, 0.44, 0.338],
#         "colorSpecular" : [0.3, 0.3, 0.3],
#         "opacity" : group!="body"
#     }

In [ ]:
###########################################
# now regular loading from parse_ascii
#########################################3
n_vertices = len(vertices)
n_faces = len(faces)

# align model
if ALIGN == "center":
    center(vertices)
elif ALIGN == "centerxz":
    centerxz(vertices)
elif ALIGN == "bottom":
    bottom(vertices)
elif ALIGN == "top":
    top(vertices)

# generate normals string
nnormal = 0
normals_string = ""
if SHADING == "smooth":
    normals_string = ",".join(generate_normal(n) for n in normals)
    nnormal = len(normals)

# extract face groups
fgs = [f['group'] for f in faces]
groups = list(set(fgs))
faceGroups = [groups.index(g) for g in fgs]

# extract morph vertices
# chance of MEMMORY ERROR here. Might need to append this to the file part by art. Or load morph seperatley
morphTargets = generate_morph_targets(morphfiles, n_vertices, infile)

# do a test to check that morphtargets are diff here

# extract morph colors
morphColors, colorFaces, materialColors = generate_morph_colors(colorfiles, n_vertices, n_faces)

# TODO
animations=[]

# generate colors string

ncolor = 0
colors_string = ""

if len(colorFaces) < len(faces):
    colorFaces = faces
    materialColors = extract_material_colors(materials, mtllib, infile)

if BAKE_COLORS:
    colors_string = ",".join(generate_color_decimal(c) for c in materialColors)
    ncolor = len(materialColors)

In [45]:
# transform morph targets names from data_targets_asym_asym-eye-5-l.target => data/targets/asym/asym-eye-5-l.target being wary of doulbe underscores we don't want to replace
# since we need the slashes in the end product but can't have them in the interim product else they would be interpreted as dirs
# and stripped. TODO proboboly best to seperate target paths and names anyway
morphTargets = [m.replace('__','+').replace('_','/').replace('+','_') for m in morphTargets]


# now make dict to export as json


TEMPLATE_FILE_ASCII = u"""{

    "metadata" :
    {
        "name"          : "%(name)s",
        "formatVersion" : 3.1,
        "sourceFile"    : "%(fname)s",
        "generatedBy"   : "OBJConverter",
        "vertices"      : %(nvertex)d,
        "faces"         : %(nface)d,
        "normals"       : %(nnormal)d,
        "colors"        : %(ncolor)d,
        "uvs"           : %(nuv)d,
        "materials"     : %(nmaterial)d,
        "morphTargets"  : %(nmorphs)d,
        "bones"         : %(nbones)d,
        "license"       : %(license)s,
        "skeleton"      : %(skeletonMetadata)s,
        "proxymesh"     : %(proxyMetadata)s
    },

    "scale" : %(scale)f,

    "materials": [%(materials)s],

    "vertices": [%(vertices)s],

    "morphTargets": [%(morphTargets)s],

    "morphColors": [%(morphColors)s],

    "normals": [%(normals)s],

    "colors": [%(colors)s],

    "uvs": [[%(uvs)s]],

    "faces": [%(faces)s],

    "bones" : %(bones)s,
    
    "skinWeights": %(skinWeights)s,
    
    "skinIndices": %(skinIndices)s,
    
    "influencesPerVertex": %(influencesPerVertex)d,

    "animations" : [%(animations)s]

}"""

text =  TEMPLATE_FILE_ASCII % {
    "name"      : get_name(outfile),
    "fname"     : os.path.basename(infile),
    "nvertex"   : len(vertices),
    "nface"     : len(faces),
    "nuv"       : len(uvs),
    "nnormal"   : nnormal,
    "ncolor"    : ncolor,
    "nmaterial" : len(materials),
    "nmorphs":len(morphTargets),
    "nbones": len(bones),
    "license": json.dumps(mh_licence.asDict()),
    "skeletonMetadata": json.dumps(skeletonMetadata),
    "proxyMetadata": json.dumps(proxyMetadata),

    "materials" : generate_materials_string(materials, mtllib, infile),

    "normals"       : normals_string,
    "colors"        : colors_string,
    "uvs"           : ",".join(generate_uv(uv) for uv in uvs),
    "vertices"      : ",".join(generate_vertex(v, TRUNCATE, SCALE) for v in vertices),

    "morphTargets"  : "\n%s\n\t" % ",\n".join(morphTargets),
    "morphColors"   : morphColors,

    "faces"     : ",".join(generate_face(f, fc) for f, fc in zip(faces, colorFaces)),

    "scale"    : SCALE,

    "bones": json.dumps(bones),

    "animations": ",".join(animations),
    
    "skinWeights": json.dumps(skinWeights),
    "skinIndices": json.dumps(skinIndices),
    "influencesPerVertex":influencesPerVertex

}



baseoutfile=outputdir.joinpath('basemodel.json')
out = open(baseoutfile, "w")
out.write(text)
out.close()

s=json.dumps(jsondata,indent=4)
json.loads(s)

print "%d vertices, %d faces, %d materials, %d morph targets. To %s" % (len(vertices), len(faces), len(materials), len(morphTargets), baseoutfile)

19158 vertices, 18486 faces, 139 materials, 0 morph targets. To /media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/output/20161117-133700_None_vae_data/basemodel.json
